In [111]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import mixture
import scipy as sp
import pydrake
from pydrake.all import (
    RandomGenerator,
    MixedIntegerRotationConstraintGenerator,
    UniformlyRandomRotationMatrix,
    IntervalBinning,
    MathematicalProgram,
    GurobiSolver,
    SolverOptions,
    RotationMatrix,
    RollPitchYaw
)
from tqdm.notebook import tqdm

In [112]:
def get_sols(R_gt, verbose=False):
    prog = MathematicalProgram()

    #R_gt = RotationMatrix(np.eye(3))

    R_dec = prog.NewContinuousVariables(3, 3, "R")
    mip_rot_gen = MixedIntegerRotationConstraintGenerator(
        approach = MixedIntegerRotationConstraintGenerator.Approach.kBilinearMcCormick,
        num_intervals_per_half_axis=2,
        interval_binning = IntervalBinning.kLogarithmic
    )
    mip_rot_gen.AddToProgram(R_dec, prog)
    prog.AddBoundingBoxConstraint(R_gt.matrix().flatten(), R_gt.matrix().flatten(), R_dec.flatten())
    
    solver = GurobiSolver()
    options = SolverOptions()
    options.SetOption(solver.id(), "LogFile", "gurobi.log")
    options.SetOption(solver.id(), "PoolSolutions", 100)
    options.SetOption(solver.id(), "PoolSearchMode", 2)

    result = solver.Solve(prog, None, options)

    N_sols = getattr(result, "num_suboptimal_solution()")()
    if verbose or N_sols > 1:
        print("R: ", result.GetSolution(R_dec), " -> # sols: ", N_sols)
    #with open("gurobi.log") as f:
    #    print(f.read())

    #for solution_k in range(getattr(result, "num_suboptimal_solution()")()):
    #    sol = result.GetSuboptimalSolution(R_dec, solution_k)
    #    print("Sol %d: %s" % (solution_k, sol))

# RPY=0 gets multiple sols
perturb = np.zeros(3)
get_sols(RotationMatrix(RollPitchYaw(perturb)), verbose=True)

# No rotation around any given axis gets multiple sols
perturb = np.array([1E-3, 0., 0.])
get_sols(RotationMatrix(RollPitchYaw(perturb)), verbose=True)

perturb = np.array([1E-3, 0., 1E-3])
get_sols(RotationMatrix(RollPitchYaw(perturb)), verbose=True)

# Rotating around all axes (no zeros in rotation matrix) gets unique sol
perturb = np.array([1E-3, 1E-3, 1E-3])
get_sols(RotationMatrix(RollPitchYaw(perturb)), verbose=True)

R:  [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]  -> # sols:  20
R:  [[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00  9.99999500e-01 -9.99999833e-04]
 [ 0.00000000e+00  9.99999833e-04  9.99999500e-01]]  -> # sols:  8
R:  [[ 9.99999500e-01 -9.99999333e-04  9.99999667e-07]
 [ 9.99999833e-04  9.99999000e-01 -9.99999333e-04]
 [ 0.00000000e+00  9.99999833e-04  9.99999500e-01]]  -> # sols:  2
R:  [[ 9.99999000e-01 -9.98999334e-04  1.00099883e-03]
 [ 9.99999333e-04  9.99999001e-01 -9.98999334e-04]
 [-9.99999833e-04  9.99999333e-04  9.99999000e-01]]  -> # sols:  1


In [113]:
# Almost all random rotations *dont* get multiple sols
# (i.e. should see no printouts)
for k in tqdm(range(100)):
    R_gt = UniformlyRandomRotationMatrix(RandomGenerator(k))
    get_sols(R_gt)

  0%|          | 0/100 [00:00<?, ?it/s]